In [21]:
import pandas as pd
from backtest_nn import *

2022-12-01 18:02:22.477497: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [54]:
soi = get_crypto_symbols()
price_data = pd.read_csv(DATA_PATH)
price_data['timestamp'] = pd.to_datetime(price_data['timestamp'])
data = pivot_price_data(price_data)
price_data = price_data[price_data['symbol'].isin(soi)]
price_data = price_data.loc[START_DATE:END_DATE]

TypeError: '<' not supported between instances of 'int' and 'Timestamp'

In [55]:
price_data.index

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            494197, 494198, 494199, 494200, 494201, 494202, 494203, 494204,
            494205, 494206],
           dtype='int64', length=494207)

In [38]:
[d for d in data.index if ~isinstance(d, pd._libs.tslibs.timestamps.Timestamp)]

[Timestamp('2022-10-01 17:26:00+0000', tz='UTC'),
 Timestamp('2022-10-01 17:27:00+0000', tz='UTC'),
 Timestamp('2022-10-01 17:28:00+0000', tz='UTC'),
 Timestamp('2022-10-01 17:29:00+0000', tz='UTC'),
 Timestamp('2022-10-01 17:30:00+0000', tz='UTC'),
 Timestamp('2022-10-01 17:31:00+0000', tz='UTC'),
 Timestamp('2022-10-01 17:32:00+0000', tz='UTC'),
 Timestamp('2022-10-01 17:33:00+0000', tz='UTC'),
 Timestamp('2022-10-01 17:34:00+0000', tz='UTC'),
 Timestamp('2022-10-01 17:35:00+0000', tz='UTC'),
 Timestamp('2022-10-01 17:36:00+0000', tz='UTC'),
 Timestamp('2022-10-01 17:37:00+0000', tz='UTC'),
 Timestamp('2022-10-01 17:38:00+0000', tz='UTC'),
 Timestamp('2022-10-01 17:39:00+0000', tz='UTC'),
 Timestamp('2022-10-01 17:40:00+0000', tz='UTC'),
 Timestamp('2022-10-01 17:41:00+0000', tz='UTC'),
 Timestamp('2022-10-01 17:42:00+0000', tz='UTC'),
 Timestamp('2022-10-01 17:43:00+0000', tz='UTC'),
 Timestamp('2022-10-01 17:44:00+0000', tz='UTC'),
 Timestamp('2022-10-01 17:45:00+0000', tz='UTC'),


In [4]:
def pivot_price_data(price_data):
    """Pivots price data from tall to wide format, 
    filters data to only after first timestamp where all symbols have non-NA
    """
    wide_data = price_data.set_index('timestamp')
    wide_data = wide_data.pivot(columns='symbol', values = 'close')
    try:
        first_valid_index = wide_data.index[wide_data.isna().mean(axis = 1)==0][0]
    except IndexError as e:
        raise IndexError(f"No record where all symbols have valid data")
    wide_data = wide_data.loc[first_valid_index:]
    return wide_data

In [9]:
data = pd.read_csv('../data/minute_crypto_data.csv')
data = pivot_price_data(data)
data = data.ffill()
data = data.bfill()

In [11]:
starting_balance = 100_000
per_day = starting_balance / data.shape[0]
symbol_per_day = per_day / data.shape[1]

### Remember: my execute function uses shares. not notional values

amounts = symbol_per_day / data

0.1288603334389988

In [19]:
data.head()

symbol,ALGO/USD,AVAX/USD,BTC/USD,DOGE/USD,ETH/USD,LINK/USD,MATIC/USD,SHIB/USD,SOL/USD
timestamp,,,,,,,,,
2022-10-01 17:26:00+00:00,0.3569,17.0565,19256.0,0.060380,1308.5,7.3770,0.762185,0.000011,32.4750
2022-10-01 17:27:00+00:00,0.3569,17.0565,19271.0,0.060380,1308.8,7.3760,0.762185,0.000011,32.5175
2022-10-01 17:28:00+00:00,0.3569,17.0565,19278.0,0.060485,1310.4,7.3860,0.762872,0.000011,32.5400
2022-10-01 17:29:00+00:00,0.3569,17.0565,19280.0,0.060524,1310.6,7.3840,0.763889,0.000011,32.5525
2022-10-01 17:30:00+00:00,0.3569,17.0565,19280.0,0.060545,1310.1,7.3915,0.763450,0.000011,32.5750


In [17]:
symbol_per_day / data

symbol,ALGO/USD,AVAX/USD,BTC/USD,DOGE/USD,ETH/USD,LINK/USD,MATIC/USD,SHIB/USD,SOL/USD
timestamp,,,,,,,,,
2022-10-01 17:26:00+00:00,0.361054,0.007555,0.000007,2.134174,0.000098,0.017468,0.169067,11588.159482,0.003968
2022-10-01 17:27:00+00:00,0.361054,0.007555,0.000007,2.134174,0.000098,0.017470,0.169067,11588.159482,0.003963
2022-10-01 17:28:00+00:00,0.361054,0.007555,0.000007,2.130433,0.000098,0.017447,0.168915,11588.159482,0.003960
2022-10-01 17:29:00+00:00,0.361054,0.007555,0.000007,2.129078,0.000098,0.017451,0.168690,11588.159482,0.003959
2022-10-01 17:30:00+00:00,0.361054,0.007555,0.000007,2.128340,0.000098,0.017434,0.168787,11588.159482,0.003956
...,...,...,...,...,...,...,...,...,...
2022-11-30 20:41:00+00:00,0.521702,0.009920,0.000008,1.236663,0.000100,0.017017,0.141341,10729.419937,0.009380
2022-11-30 20:42:00+00:00,0.523822,0.009920,0.000008,1.237851,0.000100,0.016988,0.141294,10729.419937,0.009392
2022-11-30 20:43:00+00:00,0.523822,0.009920,0.000008,1.239042,0.000100,0.016988,0.141124,10729.419937,0.009397


In [16]:
data / symbol_per_day

symbol,ALGO/USD,AVAX/USD,BTC/USD,DOGE/USD,ETH/USD,LINK/USD,MATIC/USD,SHIB/USD,SOL/USD
timestamp,,,,,,,,,
2022-10-01 17:26:00+00:00,2.769665,132.364239,149433.107040,0.468565,10154.404890,57.248028,5.914815,0.000086,252.017042
2022-10-01 17:27:00+00:00,2.769665,132.364239,149549.512140,0.468565,10156.732992,57.240268,5.914815,0.000086,252.346856
2022-10-01 17:28:00+00:00,2.769665,132.364239,149603.834520,0.469388,10169.149536,57.317871,5.920146,0.000086,252.521464
2022-10-01 17:29:00+00:00,2.769665,132.364239,149619.355200,0.469687,10170.701604,57.302351,5.928038,0.000086,252.618468
2022-10-01 17:30:00+00:00,2.769665,132.364239,149619.355200,0.469850,10166.821434,57.360553,5.924632,0.000086,252.793076
...,...,...,...,...,...,...,...,...,...
2022-11-30 20:41:00+00:00,1.916804,100.806817,132283.298864,0.808627,10030.782674,58.764399,7.075102,0.000093,106.608447
2022-11-30 20:42:00+00:00,1.909044,100.806817,132252.257504,0.807851,10023.643161,58.865283,7.077430,0.000093,106.478849
2022-11-30 20:43:00+00:00,1.909044,100.806817,132236.193600,0.807075,10023.332747,58.864507,7.085966,0.000093,106.418318
